In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf #numpy needs to be 1.23 or earlier in order for tf to import correctly


In [12]:
#make the spark dataframe df a pandas dataframe
df = pd.read_csv("income.csv")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23149 entries, 0 to 23148
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             23149 non-null  int64 
 1   workclass       23149 non-null  object
 2   education       23149 non-null  object
 3   education_num   23149 non-null  int64 
 4   marital_status  23149 non-null  object
 5   occupation      23149 non-null  object
 6   relationship    23149 non-null  object
 7   race            23149 non-null  object
 8   sex             23149 non-null  object
 9   hours_per_week  23149 non-null  int64 
 10  native_country  23149 non-null  object
 11  income          23149 non-null  object
dtypes: int64(3), object(9)
memory usage: 2.1+ MB


In [14]:
df_num = df[["age", "hours_per_week"]]
df_obj = df.drop(["age", "education_num", "hours_per_week"], axis = 1)
df_obj = df_obj.astype(str)
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_obj))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out()

# Merge one-hot encoded features and drop the originals
df = df_num.merge(encode_df,left_index=True, right_index=True)


In [15]:
#pull out our variables
y= df[["income_>50K"]]
X= df.drop(['income_<=50K', 'income_>50K'], axis = 1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X)

# Scale the data
X_scaled = X_scaler.transform(X)

In [16]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=78)

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 6


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 25)                2500      
                                                                 
 dense_5 (Dense)             (None, 12)                312       
                                                                 
 dense_6 (Dense)             (None, 6)                 78        
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 2897 (11.32 KB)
Trainable params: 2897 (11.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
543/543 [==============================] - 1s 731us/step - loss: 0.4207 - accuracy: 0.7899
Epoch 2/50
543/543 [==============================] - 0s 708us/step - loss: 0.3719 - accuracy: 0.8276
Epoch 3/50
543/543 [==============================] - 0s 716us/step - loss: 0.3561 - accuracy: 0.8319
Epoch 4/50
543/543 [==============================] - 0s 733us/step - loss: 0.3481 - accuracy: 0.8343
Epoch 5/50
543/543 [==============================] - 0s 696us/step - loss: 0.3428 - accuracy: 0.8369
Epoch 6/50
543/543 [==============================] - 0s 738us/step - loss: 0.3389 - accuracy: 0.8387
Epoch 7/50
543/543 [==============================] - 0s 736us/step - loss: 0.3360 - accuracy: 0.8407
Epoch 8/50
543/543 [==============================] - 0s 771us/step - loss: 0.3337 - accuracy: 0.8403
Epoch 9/50
543/543 [==============================] - 0s 733us/step - loss: 0.3310 - accuracy: 0.8414
Epoch 10/50
543/543 [==============================] - 0s 719us/step - loss: 0.328

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)


181/181 - 0s - loss: 0.4124 - accuracy: 0.8283 - 195ms/epoch - 1ms/step


In [21]:

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.4124058187007904, Accuracy: 0.828265368938446
